<a href="https://colab.research.google.com/github/ikfaisal/dsia/blob/master/DSIA_Lab_2_1_3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.3: 
# *Database Programming with Python and SQLite*

<a name="demo"></a>
## Using SQLite

### Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



### The `sqlite3` Command Line Utility

- useful for basic SQL tasks and databse maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14
Enter ".help" for usage hints.
sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);
insert into tbl1 values('hello!',10);
insert into tbl1 values('goodbye', 20);
select * from tbl1;`

Output:  

`hello!|10
goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands". 

Some of the more useful dot commands include:

`  
.backup ?DB? FILE      Backup DB (default "main") to FILE.
.cd DIRECTORY          Change the working directory to DIRECTORY
.clone NEWDB           Clone data into NEWDB from the existing database
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format.
.excel                 Display the output of next command in a spreadsheet
.exit                  Exit this program
.headers on|off        Turn display of headers on or off
.help                  Show this message
.import FILE TABLE     Import data from FILE into TABLE
.indexes ?TABLE?       Show names of all indexes.
.quit                  Exit this program
.read FILENAME         Execute SQL in FILENAME
.restore ?DB? FILE     Restore content of DB (default "main") from FILE
.save FILE             Write in-memory database into FILE
.tables ?TABLE?        List names of tables.`


### The `sqlite3` package

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [1]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [39]:
c.execute('DROP TABLE houses;')

# Save (commit) the changes:
conn.commit()

In [40]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')
conn.commit()

With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

> c.execute(sql_command, values)

In [41]:
# A
last_sale = (None, 4000, 5, 22, 619000)

# Execute Command

c.execute('INSERT INTO houses VALUES (?,?,?,?,?)', last_sale)

# Remember to commit the changes
conn.commit()

Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key. 

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

> c.executemany(sql_command, values)

In [42]:
# A
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

# Nb. The "?" parameters are placeholders for data that will map to the table columns during insertion;

c.executemany('INSERT INTO houses VALUES (?,?,?,?,?)', recent_sales)
# this is a security measure against SQL injection attacks:
# 

conn.commit()

Select all rows from houses

In [43]:
# A

c.execute('SELECT * FROM houses').fetchall()

[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000)]




#### Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more. 

In [44]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists:
data = genfromtxt('housing-data.csv', dtype = 'i8', 
                    delimiter = ',', skip_header = 1).tolist()

Suppose we need to put a placeholder in the first column for data that will be available later. Best practice is to insert the value `None`:

In [45]:
# prepend a None value to beginning of each sub-list:
for d in data:
    d.insert(0, None)

*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

In [46]:
# check:
print(type(data))
print(data[0:3])

<class 'list'>
[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000]]


Now we can insert each list item as a row of fields in the database.

In [47]:
# loop through data, running an INSERT on each record (i.e. sublist):
for d in data:
    c.executemany('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', data)

conn.commit()

In [48]:
c.execute('SELECT * FROM houses').fetchall()

[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000),
 (5, 2104, 3, 70, 399900),
 (6, 1600, 3, 28, 329900),
 (7, 2400, 3, 44, 369000),
 (8, 1416, 2, 49, 232000),
 (9, 3000, 4, 75, 539900),
 (10, 1985, 4, 61, 299900),
 (11, 1534, 3, 12, 314900),
 (12, 1427, 3, 57, 198999),
 (13, 1380, 3, 14, 212000),
 (14, 1494, 3, 15, 242500),
 (15, 1940, 4, 7, 239999),
 (16, 2000, 3, 27, 347000),
 (17, 1890, 3, 45, 329999),
 (18, 4478, 5, 49, 699900),
 (19, 1268, 3, 58, 259900),
 (20, 2300, 4, 77, 449900),
 (21, 1320, 2, 62, 299900),
 (22, 1236, 3, 78, 199900),
 (23, 2609, 4, 5, 499998),
 (24, 3031, 4, 21, 599000),
 (25, 1767, 3, 44, 252900),
 (26, 1888, 2, 79, 255000),
 (27, 1604, 3, 13, 242900),
 (28, 1962, 4, 53, 259900),
 (29, 3890, 3, 36, 573900),
 (30, 1100, 3, 60, 249900),
 (31, 1458, 3, 29, 464500),
 (32, 2526, 3, 13, 469000),
 (33, 2200, 3, 28, 475000),
 (34, 2637, 3, 25, 299900),
 (35, 1839, 2, 40, 349900),
 (36, 1000, 1, 5, 169900),
 (37

In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

In [49]:
import numpy as np
d1 = np.asarray([1200, 3, 15, 250000])
d1 = d1.tolist()
d1.insert(0, None)
d1
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)
conn.commit()

#### Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [50]:
c.execute('DELETE FROM houses WHERE field1 IN (52, 53)')
conn.commit()

QUIZ: What would `DELETE FROM houses` do?

This would delete the whole tabe. DANGER!!!!!

#### Filtering Rows

**1. Select Rows Where Bedrooms = 4**

In [63]:
# ANSWER

c.execute('SELECT * FROM houses WHERE Bdrms = 4').fetchall()

[(2, 2390, 4, 34, 319000),
 (9, 3000, 4, 75, 539900),
 (10, 1985, 4, 61, 299900),
 (15, 1940, 4, 7, 239999),
 (20, 2300, 4, 77, 449900),
 (23, 2609, 4, 5, 499998),
 (24, 3031, 4, 21, 599000),
 (28, 1962, 4, 53, 259900),
 (37, 2040, 4, 75, 314900),
 (39, 1811, 4, 24, 285900),
 (42, 2132, 4, 28, 345000),
 (43, 4215, 4, 66, 549000),
 (44, 2162, 4, 43, 287000),
 (47, 2567, 4, 57, 314000),
 (50, 1852, 4, 64, 299900),
 (56, 3000, 4, 75, 539900),
 (57, 1985, 4, 61, 299900),
 (62, 1940, 4, 7, 239999),
 (67, 2300, 4, 77, 449900),
 (70, 2609, 4, 5, 499998),
 (71, 3031, 4, 21, 599000),
 (75, 1962, 4, 53, 259900),
 (84, 2040, 4, 75, 314900),
 (86, 1811, 4, 24, 285900),
 (89, 2132, 4, 28, 345000),
 (90, 4215, 4, 66, 549000),
 (91, 2162, 4, 43, 287000),
 (94, 2567, 4, 57, 314000),
 (97, 1852, 4, 64, 299900),
 (103, 3000, 4, 75, 539900),
 (104, 1985, 4, 61, 299900),
 (109, 1940, 4, 7, 239999),
 (114, 2300, 4, 77, 449900),
 (117, 2609, 4, 5, 499998),
 (118, 3031, 4, 21, 599000),
 (122, 1962, 4, 53, 25

**2. Run a query to calculate the average floor area and price of each size of house (i.e. by number of bedrooms):**

In [64]:
# ANSWER
results = c.execute("SELECT bdrms, AVG(sqft) AS avg_sqft, AVG(price) AS avg_price FROM houses GROUP BY bdrms")
results.fetchall()

[(1, 1000.0, 169900.0),
 (2, 1496.5, 280866.6666666667),
 (3, 1837.937074829932, 326192.862244898),
 (4, 2400.412746585736, 377361.09104704094),
 (5, 4468.041666666667, 698214.5833333334)]

### Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behaviour or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [65]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

### Writing data into a database

Data in Pandas can be loaded into a relational database. 

If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [67]:
# Nb. Use low_memory=False to ensure that type inference does not fail due to buffered processing of input:
data = pd.read_csv('housing-data.csv', low_memory = False)
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [78]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)

Run a query to get the average price of each house size from this table:

In [81]:
#ANSWER

c.execute('SELECT bdrms, AVG (price) FROM houses_pandas GROUP BY bdrms').fetchall()



[(1, 169900.0),
 (2, 280866.6666666667),
 (3, 326403.92),
 (4, 377449.78571428574),
 (5, 699900.0)]

## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database? 

ANS - when there are advanced manipulations which i would like to do.

2. When would you want to use Pandas on a dataset retrieved from a database?

ANS - when there are advanced manipulations other than the basic operators available in sql which i would like to do

<a name="guided-practice"></a>


# Reference: SQL Syntax 


#### SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

### AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate funtions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

### Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

#### 1. Read CSV into DataFrame

In [82]:
import pandas as pd
import sqlite3
from pandas.io import sql

In [197]:
# Reading CSV to Dataframe
orders = pd.read_csv('P12-ListOfOrders.csv', encoding = 'utf-8')
orders_break_down =  pd.read_csv('P12-OrderBreakdown.csv', encoding = 'utf-8')

In [198]:
orders.head()

,Order ID,Order Date,Customer Name,Country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [85]:
orders_break_down.head()

,Order ID,Product Name,Discount,Sales,Quantity,Category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


#### 2. Replace Space with Underscore in Column Names

In [86]:
# ANSWER
orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]

In [87]:
# ANSWER
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]

#### 3. Check DataTypes 

In [88]:
# ANSWER
orders.dtypes

order_id         object
order_date       object
customer_name    object
country          object
dtype: object

In [89]:
# ANSWER
orders_break_down.dtypes

order_id         object
product_name     object
discount        float64
sales           float64
quantity          int64
category         object
dtype: object

#### 4. Save these two dataframes as a table in sqlite

In [91]:
# Establishing Local DB connection
db_connection = sqlite3.connect('eshop.db.sqlite')

# Save these two dataframes as a table in sqlite

orders.to_sql(name = 'orders', con = db_connection, if_exists = 'replace', index = False)
orders_break_down.to_sql(name = 'orders_break_down', con = db_connection, if_exists = 'replace', index = False)

#### 5. Select Number of Orders for Each Customer

We can use the `sql` function in `pandas.io` to run SQL queries to the database with the `.read_sql()` method. Here's an example:

In [92]:
# Select first 10 rows of all columns in orders table

query = """
        SELECT * 
        FROM orders 
        LIMIT 10;
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France
5,ES-2011-5433855,2011-01-07,Dave Poirier,France
6,IT-2011-4546695,2011-01-08,Darren Powers,France
7,ES-2011-1138719,2011-01-11,Eric Murdock,Italy
8,ES-2011-1466305,2011-01-11,Mick Brown,Austria
9,ES-2011-4359424,2011-01-11,Dorothy Dickinson,Spain


In [98]:
# Select Number of Orders for Each Customer
sql_string = '''
    SELECT 
        customer_name, 
        COUNT(order_id) AS order_count 
    FROM orders 
    GROUP BY customer_name 
    ORDER BY order_count DESC
'''

customers = sql.read_sql(sql_string, con = db_connection)
customers.head()

,customer_name,order_count
0,John Grady,13
1,Joel Jenkins,12
2,Aaron Smayling,12
3,Yoseph Carroll,11
4,Sarah Brown,11


#### 6. Select Number of Customers for Each Country

In [94]:
# Select Number of Customers for Each Country

sql_string = '''
    SELECT 
        country, 
        COUNT(DISTINCT customer_name) AS Count 
    FROM orders 
    GROUP BY country 
    ORDER BY Count DESC
'''

customers = sql.read_sql(sql_string, con = db_connection)
customers.head()

,country,Count
0,France,568
1,Germany,505
2,United Kingdom,473
3,Italy,351
4,Spain,306


#### 7.A Select discount, sales, quantity for Each Order from orders_break_down Table

In [107]:
#  Select discount, sales, quantity for Each Order from orders_break_down Table

sql_string = '''
    SELECT 
        discount, 
        sales,
        quantity
        
    FROM orders_break_down 
   
    
'''

customers = sql.read_sql(sql_string, con = db_connection)
customers.head()

,discount,sales,quantity
0,0.5,45.0,3
1,0.0,854.0,7
2,0.0,140.0,3
3,0.5,27.0,2
4,0.5,17.0,2


#### 7.B Select discount, sales, quantity, total price for Each Order from orders_break_down Table

> Total Price = sales * quantity - discount

In [109]:
# Select discount, sales, quantity, total price for Each Order from orders_break_down Table

sql_string = '''
    SELECT 
        discount, 
        sales,
        quantity, 
        sales*quantity-discount AS total_price
        
    FROM orders_break_down 
   
    
'''

customers = sql.read_sql(sql_string, con = db_connection)
customers.head()

,discount,sales,quantity,total_price
0,0.5,45.0,3,134.5
1,0.0,854.0,7,5978.0
2,0.0,140.0,3,420.0
3,0.5,27.0,2,53.5
4,0.5,17.0,2,33.5


#### 7.C Select All Orders from orders_break_down Table Where Total Price Greater Than 100

In [116]:
# Select All Orders from orders_break_down Table Where Total Price Greater Than 100


sql_string = '''
    SELECT 
        *
        , (sales * quantity - discount) AS total_price
    FROM orders_break_down 
    WHERE total_price > 100
    ORDER BY total_price ASC

'''

customers = sql.read_sql(sql_string, con = db_connection)
customers.head(10)

,order_id,product_name,discount,sales,quantity,category,total_price
0,IT-2011-2668777,"Binney & Smith Canvas, Blue",0.5,51.0,2,Office Supplies,101.5
1,IT-2011-1165265,"Cardinal Binding Machine, Durable",0.5,51.0,2,Office Supplies,101.5
2,ES-2012-1763830,"Elite Scissors, High Speed",0.5,34.0,3,Office Supplies,101.5
3,IT-2012-4099397,"Sanford Canvas, Blue",0.5,51.0,2,Office Supplies,101.5
4,ES-2013-4250046,"Cardinal Binding Machine, Durable",0.5,51.0,2,Office Supplies,101.5
5,ES-2014-5784412,"Okidata Calculator, White",0.5,51.0,2,Technology,101.5
6,IT-2014-5393609,"Tenex Lockers, Single Width",0.5,102.0,1,Office Supplies,101.5
7,ES-2012-5499117,"Fellowes Box, Wire Frame",0.4,34.0,3,Office Supplies,101.6
8,ES-2014-3096832,"Fellowes Box, Wire Frame",0.4,34.0,3,Office Supplies,101.6
9,ES-2011-4665208,"Avery Binder Covers, Durable",0.1,34.0,3,Office Supplies,101.9


#### 8. Select All Customer And The Product They Have Bought

In [121]:
# Select All Customer And The Product They Have Bought

sql_string = '''
    SELECT 
        orders.order_id
        , orders.customer_name
        , orders_break_down.product_name
    FROM orders
    JOIN orders_break_down ON orders.order_id = orders_break_down.order_id
''' 
customers = sql.read_sql(sql_string, con = db_connection)
customers.head()

,order_id,customer_name,product_name
0,IT-2011-3647632,Eugene Moren,"Enermax Note Cards, Premium"
1,ES-2011-4869686,Dorothy Dickinson,"Dania Corner Shelving, Traditional"
2,ES-2011-4939443,Arthur Prichep,"Binney & Smith Sketch Pad, Easy-Erase"
3,IT-2011-2942451,Grant Thornton,"Boston Markers, Easy-Erase"
4,IT-2011-2942451,Grant Thornton,"Eldon Folders, Single Width"


#### 9.A Select Number of 'Furniture' Orders For Each Country 

In [137]:
# Select Number of 'Furniture' Orders For Each Country


sql_string = '''
    SELECT 
        orders.country
        , COUNT(orders_break_down.order_id) AS Count           
    FROM orders
    JOIN orders_break_down
    ON orders.order_id = orders_break_down.order_id
    WHERE orders_break_down.category = 'Furniture'
    GROUP BY orders.country
'''

order_list = sql.read_sql(sql_string, con = db_connection)

order_list.head()

,country,Count
0,Austria,40
1,Belgium,20
2,Denmark,9
3,Finland,16
4,France,299


#### 9.B Select Number of 'Furniture' Orders For The Country Denmark

In [141]:
# Select Number of 'Furniture' Orders For The Country Denmark


sql_string = '''
    SELECT 
        COUNT(orders_break_down.order_id) AS Count           
    FROM orders
    JOIN orders_break_down
    ON orders.order_id = orders_break_down.order_id
    WHERE orders_break_down.category = 'Furniture'
    AND orders.country = 'Denmark'
    
'''

order_list = sql.read_sql(sql_string, con = db_connection)

order_list.head()


,Count
0,9


#### 10. Select Total Sales With Discount and Without Discount for Each Country

In [168]:
# Select Total Sales With Discount and Without Discount for Each Country



sql_string = '''
    SELECT 
    orders.country,orders_break_down.discount,
        SUM(orders_break_down.sales)  AS total_sales     
    FROM orders_break_down
    JOIN orders
    ON orders.order_id = orders_break_down.order_id
           GROUP BY orders_break_down.discount
'''

order_list = sql.read_sql(sql_string, con = db_connection)

order_list.head()

,country,discount,total_sales
0,United Kingdom,0.00,1232868.0
1,France,0.10,572657.0
2,France,0.15,191864.0
3,Spain,0.20,36842.0
4,United Kingdom,0.30,4952.0


#### 11.A Select Total Quantity, Total Sales for Each Country

In [147]:
# Select Total Quantity, Total Sales for Each Country


sql_string = '''
    SELECT
        orders.country
        , SUM(sales) AS total_sales
        , SUM(quantity) AS total_quantity       
    FROM orders
    JOIN orders_break_down ON orders.order_id = orders_break_down.order_id
    GROUP BY orders.country
'''

order_list = sql.read_sql(sql_string, con = db_connection)

order_list.head()

,country,total_sales,total_quantity
0,Austria,79382.0,973
1,Belgium,42320.0,532
2,Denmark,7763.0,204
3,Finland,20702.0,201
4,France,609683.0,7329


#### 11.B Select Top 3 Country Based on Sales

In [170]:
# Select Top 3 Country Based on Sales


order_list.head(3)

,country,discount,total_sales
0,United Kingdom,0.00,1232868.0
1,France,0.10,572657.0
2,France,0.15,191864.0


#### 11.C Select Bottom 3 Country Based On Quantities

In [171]:
# Select Bottom 3 Country Based On Quantities

order_list.tail(3)

,country,discount,total_sales
11,Netherlands,0.70,3658.0
12,United Kingdom,0.80,139.0
13,Germany,0.85,555.0


#### 12. Select Average Sales By Categroy For The Country 'France'

In [187]:
# Select Average Sales By Categroy For The Country 'France'


sql_string = '''
    SELECT
        orders.country,
        orders_break_down.category,
        AVG(sales) AS avg_sales
              
    FROM orders
    JOIN orders_break_down ON orders.order_id = orders_break_down.order_id
   WHERE orders.country = 'France'
   GROUP BY orders_break_down.category
'''

sales_list = sql.read_sql(sql_string, con = db_connection)

sales_list.head()

,country,category,avg_sales
0,France,Furniture,582.314381
1,France,Office Supplies,167.217709
2,France,Technology,595.145078


#### 13. Select Country, Category and Total Sales Where Average Total Sales is The Highest

In [205]:
# Select Country, Category and Total Sales Where Average Total Sales is The Highest



sql_string = '''
    SELECT
        orders.country,
        orders_break_down.category,
        SUM(orders_break_down.sales) AS total_sales
              
    FROM orders
    JOIN orders_break_down ON orders.order_id = orders_break_down.order_id
   
   
'''

sales_list = sql.read_sql(sql_string, con = db_connection)

sales_list.head(1)

,country,category,total_sales
0,Sweden,Office Supplies,2348482.0


### JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams. 
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

### ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  
- [SQL zoo](http://www.sqlzoo.net)  Great for learning syntax



---



---



> > > > > > > > > © 2019 Institute of Data


---



---



